## dev环境数据拉取

In [1]:
import json
import time

import requests
import xmltodict

from connection import ConnectionTask

c=ConnectionTask()
union_id = 61713180


def get_all_fconfig():
    url = "http://61.228.cn/unionOrder/getAllFconfig?unionId={}".format(union_id)
    headers = {"Action": "getAllFconfig"}
    resp_data = requests.post(url=url, headers=headers).text
    resp_data = xmltodict.parse(resp_data)['Response']['getFconfigRsp']['fconfigInfo']
    c.smdb.piaowu_yongle.delete_many({})
    datas_cleanse('piaowu_yongle')
    for city in resp_data:
        city_code = city['citycode']
        get_product_info(city_code)
    return 'ok'


def get_product_info(city_code):
    headers = {"Action": "getAllFconfig"}
    url = "http://61.228.cn/upload/DataXml/{}/{}.xml".format(union_id, city_code)
    resp_data = requests.get(url, headers=headers).text.replace('http://', 'https://')
    try:
        resp_data = xmltodict.parse(resp_data)['products']
        if not resp_data['subjects']:
            return city_code, None

        products = resp_data['product']
        if not isinstance(products, list):
            return city_code, 'no list'
    except (KeyError, xmltodict.expat.ExpatError) as why:
        return city_code, 'err'

    c.smdb.piaowu_yongle.delete_many({'playCityCode': city_code})

    for product in products:
        product_id = product.pop('productId')
        # 字段ticketTimeList 数据整理
        ticket_time_list = product.pop('ticketTimeList')
        if isinstance(ticket_time_list['ticketTimeInfo'], dict):
            # ticketTimeList.ticketTimeInfo must be a list
            ticket_time_list['ticketTimeInfo'] = [ticket_time_list['ticketTimeInfo']]
        for index, ticket_time_info in enumerate(ticket_time_list['ticketTimeInfo']):
            if 'tickSetInfoList' not in ticket_time_info:
                continue
            tick_set_info_list = ticket_time_info['tickSetInfoList']
            if isinstance(tick_set_info_list['tickSetInfo'], dict):
                # ticketTimeList.ticketTimeInfo[index].tickSetInfoList.tickSetInfo must be a list
                tick_set_info_list['tickSetInfo'] = [tick_set_info_list['tickSetInfo']]
                ticket_time_list['ticketTimeInfo'][index]['tickSetInfoList'] = tick_set_info_list
        product['ticketTimeList'] = ticket_time_list
        c.smdb.piaowu_yongle.update_one({"productId": product_id}, {"$set": product}, upsert=True)
    return city_code, 'ok'

def datas_cleanse(database):
    resp = requests.get("http://192.168.0.6:8024/mongodb_query/delete/{}".format(database)).json()
    if resp['code'] != 200:
        raise KeyError
    return 'DeleteDatabase: {}'.format(database)

In [ ]:
get_all_fconfig()

In [ ]:
datas_cleanse('piaowu_yongle')

## 接口测试脚本

In [ ]:
union_id= 61713180
url="http://61.228.cn/unionOrder/getAllFconfig?unionId={}".format(union_id)
headers = {"Action":"getAllFconfig"}
resp=requests.post(url=url,headers=headers)
resp_data=resp.text
# xmltodict.parse(resp_data)['Response']['getFconfigRsp']['fconfigInfo']

##  查询所有分站接口（getAllFconfig）

In [ ]:
# url="http://61.228.cn/unionOrder/getAllFconfig?unionId={}".format(union_id)
# headers = {"Action":"getAllFconfig"}
# resp_data=requests.post(url=url,headers=headers).text
# resp_data = xmltodict.parse(resp_data)['Response']['getFconfigRsp']['fconfigInfo']
# city_data={}
# for city in resp_data:
#     city_data[city['playCity']]=city['citycode']
# city_data
#     xmlt_data = (json.loads(
#         (json.dumps(xmlt_data, ensure_ascii=False, indent=2)).replace('@', '')))

In [ ]:
# 正式环境数据
def get_product_info(city_code):
    headers = {"Action": "getAllFconfig"}
    url = "http://lm.228.cn/upload/DataXml/62485393/{}.xml".format(city_code)
    resp_data = requests.get(url, headers=headers).text.replace('http://', 'https://')
    
    resp_data = xmltodict.parse(resp_data)['products']
    return resp_data
    products = resp_data['product']
    return products
dict(get_product_info('0081'))

In [4]:
citys=get_all_fconfig()
yongle_citys={}
for city in citys:
    yongle_citys[city['playCity']]=city['citycode']

AttributeError: 'ConnectionTask' object has no attribute 'pro'

In [2]:
YONGLE_UNIONID = 62485393
def get_all_fconfig():
    url = "http://lm.228.cn/unionOrder/getAllFconfig?unionId={}".format(YONGLE_UNIONID)
    headers = {"Action": "getAllFconfig"}
    resp_data = requests.post(url=url, headers=headers).text
    resp_data = xmltodict.parse(resp_data)['Response']['getFconfigRsp']['fconfigInfo']
    for city in resp_data:
        city_code = city['citycode']
#         get_product_info.apply_async(args=[city_code], queue=conf.MQ_YONGLE)
    return resp_data
# get_all_fconfig()

In [ ]:
# 订单轮训接口 根据联盟id和永乐订单id、时间查询订单接口（getOrderList）
YONGLE_UNIONID = 62485393
from bs4 import BeautifulSoup

def order_status_polling(data):
    url = "http://lm.228.cn/unionOrder/findOrderByUnionidAndOrderid"
    headers = {"Action": "findOrderByUnionidAndOrderid"}
    payload={
        'unionId':YONGLE_UNIONID,
        'unionOrderId':data['orderId']
    }
    resp = requests.get(url=url, headers=headers,params=payload).text
    resp_data = BeautifulSoup(resp, "html5lib")
    return resp_data.find('orderstarus').text

order_status_polling({'orderId':'blm102407380811820'})